Mine

In [5]:
import torch
import torchtext
from torch import nn
from TorchCRF import CRF

In [ ]:
class BiLSTM_CRF(nn.Module):
    def __init__(self) -> None:
        super().__init__()

ChatGPT example

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.legacy.datasets import SequenceTaggingDataset
from torchtext.legacy.data import Field, BucketIterator

# 定义模型超参数
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
BATCH_SIZE = 32
EPOCHS = 10

# 定义数据集和数据预处理参数
TEXT = Field(lower=True)
TAGS = Field(unk_token=None)
fields = (("text", TEXT), ("tags", TAGS))
train_data, valid_data, test_data = SequenceTaggingDataset.splits(
    path="data",
    train="eng.train",
    validation="valid.txt",
    test="eng.testa",
    fields=fields,
)
TEXT.build_vocab(train_data)
TAGS.build_vocab(train_data)
VOCAB_SIZE = len(TEXT.vocab)
TAGS_SIZE = len(TAGS.vocab)


In [ ]:
class BiLSTM_CRF(nn.Module):
    def __init__(self, vocab_size, tag_size, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim, tag_size)
        self.crf = nn.CRF(tag_size)
    
    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out)
        scores = self.crf.decode(out)
        return scores


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM_CRF(VOCAB_SIZE, TAGS_SIZE, EMBEDDING_DIM, HIDDEN_DIM).to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=TAGS.vocab.stoi[TAGS.pad_token])

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
)

for epoch in range(EPOCHS):
    epoch_loss = 0
    model.train()
    for batch in train_iterator:
        optimizer.zero_grad()
        text = batch.text
        tags = batch.tags
        scores = model(text)
        loss = criterion(scores.view(-1, TAGS_SIZE), tags.view(-1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    avg_epoch_loss = epoch_loss / len(train_iterator)
    print(f"Epoch {epoch+1} train loss: {avg_epoch_loss:.4f}")


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

model.eval()
true_tags = []
pred_tags = []
with torch.no_grad():
    for batch in test_iterator:
        text = batch.text
        tags = batch.tags
        scores = model(text)
        for i in range(len(text)):
            true_tags.extend([TAGS.vocab.itos[tags[j][i].item()] for j in range(len(tags))])
            pred_tags.extend([TAGS.vocab.itos[scores[j][i].item()] for j in range(len(scores))])
accuracy = sum([1 for i in range(len(true_tags)) if true_tags[i] == pred_tags[i]]) / len(true_tags)
precision = precision_score(true_tags, pred_tags, average="macro")
recall = recall_score(true_tags, pred_tags, average="macro")
f1 = f1_score(true_tags, pred_tags, average="macro")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 score: {f1:.4f}")
